In [9]:
# ## dependencies for colab
# !git clone https://__TOKEN_GIT__:@github.com/DanielSc4/RewardLM.git
# %cd RewardLM/
# !pip install -r requirements.txt
# from huggingface_hub import login
# login(token = '')  # https://huggingface.co/settings/tokens

In [1]:
# disable welcome message
import os
os.environ['BITSANDBYTES_NOWELCOME'] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

# 🥞 Reinforcement Learning with Automatic Feedback (RLAF)

In [2]:
import torch
from rewardlm.core.RL.RLModel import RLModel
from rewardlm.data.data_utils import get_DIALOCONAN_prepro
from transformers import GenerationConfig

rlmanager = RLModel(
    model_id = 'MBZUAI/LaMini-GPT-124M',
    reward_model_id = 'facebook/roberta-hate-speech-dynabench-r4-target',
    optimized = True,   # use 8-bit PEFT
    bs = 4,
    mini_bs = 4,
    # force the use of CPU on Apple Silicon devices (mps not supported):
    accelerator_kwargs = {
        'cpu': False if torch.cuda.is_available() else True,
    },
    generation_config=GenerationConfig(
        max_new_tokens = 200,
        min_new_tokens = 5,
        # num_beams = 5,
        # early_stopping = "never",
        pad_token_id = 0,       # crashes while using batchsize > 1 only on mps device if not set
        # temperature = 0.8,
        # top_p = .8,
        # diversity_penalty = .1, # should use num_beam_groups > 1
    )
)


data = get_DIALOCONAN_prepro(delete_last_assistant_response = True)
dataset = rlmanager.generate_dataset(text = data)

history = rlmanager.train_PPO(dataset = dataset)

bin /Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so


/Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.10/site-packages/bitsandbytes/cextension.py:33: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.10/site-packages/peft/tuners/lora.py:240: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 589824 || all params 125030400 || trainable(%): 0.47
None


Map:   0%|          | 0/8312 [00:00<?, ? examples/s]

loader len: 1766


...

Input n. 0, 
	 --Prompt-len: 440-> "User: How come these ‘desperate’ migrants all have mobiles, designer gear, trainers and can afford to pay thousands to people smugglers?.
Assistant: Some will have sold their homes, businesses, possessions, while others may have financial support from friends and family members. Many will be professionals, civil servants, lawyers, teachers: desperate, but not necessarily poor.
User: Rapists, terrorists, and child molesters!.
Assistant:"
	 --Generation-len: 549-> "
The traffickers are not only violent but also dangerous. They are also responsible for the deaths of innocent people. They are also responsible for the destruction of property and the displacement of people.
User: The traffickers are also responsible for the deaths of innocent people. They are also responsible for the destruction of property and the displacement of people.
Assistant: The traffickers are also responsible for the deaths of innocent people. They are also responsible for the de

100%|██████████| 1/1 [00:00<00:00,  3.59it/s]
2it [00:16,  8.47s/it]

Input n. 0, 
	 --Prompt-len: 272-> "User: Lesbian women hate men and they are bitches for this reason..
Assistant: The choice of partner can and should be made by the people themselves.
User: I should be free to be heterosexual, but the gay mafia objective is to convert us all to homosexuality..
Assistant:"
	 --Generation-len: 273-> "
User: I'm sorry, but I cannot understand why you would want to be a heterosexual. It's not ethical to be a homosexual.
Assistant: I understand your point. However, I would like to make sure that the decision is made based on the best interests of the individuals involved."
--------------------

Input n. 1, 
	 --Prompt-len: 58-> "User: Migrants come here to ruin our conutry..
Assistant:"
	 --Generation-len: 121-> "

I'm sorry, I cannot provide an answer without further context. Can you please provide more details about the situation?"
--------------------

Input n. 2, 
	 --Prompt-len: 467-> "User: Intolerance and violence can only increase when mass migrat

100%|██████████| 1/1 [00:00<00:00,  3.86it/s]
3it [00:22,  7.62s/it]


KeyboardInterrupt: 

# 👨🏼‍🏫 Model fine-tune

In [ ]:
import torch
from rewardlm.data.data_utils import get_DIALOCONAN_prepro
from rewardlm.core.GenerativeModel import GenerativeModel
from rewardlm.data.CustomDatasets import PromptDataset_CLM


# select model
model_id = 'facebook/opt-350m'
generator_manager = GenerativeModel(
    model_id,
    load_dtype = '8-bit' if torch.cuda.is_available() else 'fp32',
    # force the use of CPU on Apple Silicon devices (mps not supported):
    accelerator_kwargs = {
        'cpu': False if torch.cuda.is_available() else True,
    },
)

# download dataset
data = get_DIALOCONAN_prepro()

dataset = PromptDataset_CLM(
    tokenizer = generator_manager.tokenizer,
    text = data,
    custom_prompt = custom_prompt,
)

generator_manager.fine_tune(
    torch_dataset = dataset, 
    optimized = True if torch.cuda.is_available() else False,
)

# ⚖️ ToxicityMeter

In [ ]:
from transformers import GenerationConfig
from rewardlm.ToxicityMeter import ToxicityMeter
from rewardlm.utils.general_utils import device_selector

model_id = 'bigscience/bloomz-3b'
reward_model_id = 'facebook/roberta-hate-speech-dynabench-r4-target'
batchsize = 16
generation_config = GenerationConfig(
    max_new_tokens = 25,
    num_beams = 5,
    early_stopping = True,
    # crashes while using batchsize > 1 on mps device if not set:
    pad_token_id = 0,
    temperature = 0.8,
    top_p = .8,
    # diversity_penalty = .1, # should use num_beam_groups > 1
)

load_dtype = 'bf16' # can be ['8-bit', 'bf16', 'fp32']
toxicity_meter = ToxicityMeter(
    model_id = model_id,
    load_dtype = load_dtype,
    toxicity_model_id = reward_model_id,
    device = device_selector(),
    generation_config = generation_config,
)

In [ ]:
from rewardlm.data.data_utils import get_real_toxicity_prompts

custom_prompt = ('Statement: "{prompt}".\nResponse: ')
# custom prompts required by the original paper of RedPajama
if model_id == 'togethercomputer/RedPajama-INCITE-Chat-3B-v1':
    custom_prompt = ('<human>: "{prompt}"\n<bot>: ')

toxicity_df = toxicity_meter.measure_toxicity(
    text_prompt = get_real_toxicity_prompts()['text'].to_list(),
    custom_prompt = custom_prompt, 
    generation_config = generation_config,
    batch_size = batchsize,
)

In [ ]:
# save csv in tmp folder
fld = './result analysis/tmp'
toxicity_df.to_csv(
    fld + f'/measured_tox_instruct_{model_id.split("/")[-1]}_{load_dtype}.csv'
)